In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import library

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_curve, auc,precision_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from imblearn.over_sampling import SMOTE,ADASYN,BorderlineSMOTE,SVMSMOTE,KMeansSMOTE
from imblearn.under_sampling import NearMiss,EditedNearestNeighbours,TomekLinks
from imblearn.combine import SMOTEENN,SMOTETomek


from warnings import filterwarnings
filterwarnings('ignore')
import gc

In [ ]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [ ]:
files = [
#     '../input/ieee-fraud-detection/test_identity.csv', 
#          '../input/ieee-fraud-detection/test_transaction.csv',
         '../input/ieee-fraud-detection/train_identity.csv',
         '../input/ieee-fraud-detection/train_transaction.csv',
         '../input/ieee-fraud-detection/sample_submission.csv']

In [ ]:
%%time
def load_data(file):
    return pd.read_csv(file)

with multiprocessing.Pool() as pool:
#     test_id, test_tr, train_id, train_tr, sub = pool.map(load_data, files)
    train_id, train_tr, sub = pool.map(load_data, files)

In [ ]:
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
# test = pd.merge(test_tr, test_id, on='TransactionID', how='left')

# del test_id, test_tr, train_id, train_tr
del train_id, train_tr
gc.collect()

In [ ]:
useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                   'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                   'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
len(useful_features)

In [ ]:
cols_to_drop = [col for col in train.columns if col not in useful_features]
cols_to_drop.remove('isFraud')
cols_to_drop.remove('TransactionID')
cols_to_drop.remove('TransactionDT')

In [ ]:
print('{} features are going to be dropped for being useless'.format(len(cols_to_drop)))

train = train.drop(cols_to_drop, axis=1)
# test = test.drop(cols_to_drop, axis=1)

In [ ]:
# New feature - decimal part of the transaction amount
train['TransactionAmt_decimal'] = ((train['TransactionAmt'] - train['TransactionAmt'].astype(int)) * 1000).astype(int)
# test['TransactionAmt_decimal'] = ((test['TransactionAmt'] - test['TransactionAmt'].astype(int)) * 1000).astype(int)

# Count encoding for card1 feature. 
# Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
# train['card1_count_full'] = train['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
# test['card1_count_full'] = test['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))

# https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
# test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)
train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24
# test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    train[feature] = train[f1].astype(str) + '_' + train[f2].astype(str)
#     test[feature] = test[f1].astype(str) + '_' + test[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train[feature].astype(str).values))
    train[feature] = le.transform(list(train[feature].astype(str).values))
#     test[feature] = le.transform(list(test[feature].astype(str).values))
    
# for feature in ['id_34', 'id_36']:
#     if feature in useful_features:
#         # Count encoded for both train and test
#         train[feature + '_count_full'] = train[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
#         test[feature + '_count_full'] = test[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
        
for feature in ['id_01', 'id_31', 'id_33', 'id_35', 'id_36']:
    if feature in useful_features:
        # Count encoded separately for train and test
        train[feature + '_count_dist'] = train[feature].map(train[feature].value_counts(dropna=False))
#         test[feature + '_count_dist'] = test[feature].map(test[feature].value_counts(dropna=False))

In [ ]:
for col in tqdm_notebook(train.columns):
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
#         test[col] = le.transform(list(test[col].astype(str).values))   

In [ ]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']
# test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)

In [ ]:
# by https://www.kaggle.com/dimartinot
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
# sample_test = clean_inf_nan(sample_test)

X = X.fillna(-1)

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                 test_size=0.2,
                                                 random_state=42)

In [ ]:
del train
del X, y
gc.collect()

In [ ]:
scaler = StandardScaler()

# Fit only to the training data
scaler.fit(X_train)

# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
%%time
# build the lightgbm model
clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
clf.fit(X_train, y_train)

In [ ]:
# %%time
# dt_clf = DecisionTreeClassifier(random_state=1)
# dt_clf.fit(X_train,y_train)


In [ ]:
%%time
y_pred = clf.predict(X_test)

In [ ]:
print('The ACC of the is:', metrics.accuracy_score(y_test, y_pred))
print('The Recall of the is:', metrics.recall_score(y_test, y_pred))
print('The Precision of the is:', metrics.precision_score(y_test, y_pred))
print('The F-score of the is:', metrics.f1_score(y_test, y_pred))
print("confusion_matrix:")
print(confusion_matrix(y_test, y_pred))
fpr, tpr, thresh = roc_curve(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred)
print("auc_score:",auc_score)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# above is a benchmark 

# Decomposition

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap,LocallyLinearEmbedding,TSNE

In [ ]:
%%time
pca = PCA(n_components = 0.95)
pca.fit(X_train)
X_train_de = pca.transform(X_train)
X_test_de = pca.transform(X_test)

In [ ]:
print(X_train_de.shape, y_train.shape)
print(y_train.value_counts())

In [ ]:
%%time
# dt_clf_de = DecisionTreeClassifier(random_state=1)
# dt_clf_de.fit(X_train_de,y_train)
clf_de = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
clf_de.fit(X_train_de, y_train)

In [ ]:
%%time
y_pred_de = clf_de.predict(X_test_de)

In [ ]:
print('The ACC of the is:', metrics.accuracy_score(y_test, y_pred_de))
print('The Recall of the is:', metrics.recall_score(y_test, y_pred_de))
print('The Precision of the is:', metrics.precision_score(y_test, y_pred_de))
print('The F-score of the is:', metrics.f1_score(y_test, y_pred_de))
print("confusion_matrix:")
print(confusion_matrix(y_test, y_pred_de))
# fpr, tpr, thresh = roc_curve(y_test, y_pred_de)
auc_score_de = roc_auc_score(y_test, y_pred_de)
print("auc_score_de:",auc_score_de)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_de))

# Resample

In [ ]:
%%time
X_smo,y_smo = SMOTE(sampling_strategy='minority').fit_resample(X_train,y_train)

In [ ]:
print(X_smo.shape,y_smo.shape)

In [ ]:
%%time
# dt_clf_re = DecisionTreeClassifier(random_state=1)
# dt_clf_re.fit(X_smo,y_smo)
clf_re = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
clf_re.fit(X_smo, y_smo)

In [ ]:
%%time
y_pred_re = clf_re.predict(X_test)

In [ ]:
print('The ACC of the is:', metrics.accuracy_score(y_test, y_pred_re))
print('The Recall of the is:', metrics.recall_score(y_test, y_pred_re))
print('The Precision of the is:', metrics.precision_score(y_test, y_pred_re))
print('The F-score of the is:', metrics.f1_score(y_test, y_pred_re))
print("confusion_matrix:")
print(confusion_matrix(y_test, y_pred_re))
# fpr, tpr, thresh = roc_curve(y_test, y_pred_de)
auc_score_re = roc_auc_score(y_test, y_pred_re)
print("auc_score_re:",auc_score_re)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_re))

# Both Decomposition and Resample

In [ ]:
X_de_smo,y_de_smo = SMOTE(sampling_strategy='minority').fit_resample(X_train_de,y_train)

In [ ]:
print(X_de_smo.shape,y_de_smo.shape)

In [ ]:
%%time
# dt_clf_re = DecisionTreeClassifier(random_state=1)
# dt_clf_re.fit(X_smo,y_smo)
clf_de_re = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
clf_de_re.fit(X_de_smo, y_de_smo)

In [ ]:
%%time
y_pred_de_re = clf_de_re.predict(X_test_de)

In [ ]:
print('The ACC of the is:', metrics.accuracy_score(y_test, y_pred_de_re))
print('The Recall of the is:', metrics.recall_score(y_test, y_pred_de_re))
print('The Precision of the is:', metrics.precision_score(y_test, y_pred_de_re))
print('The F-score of the is:', metrics.f1_score(y_test, y_pred_de_re))
print("confusion_matrix:")
print(confusion_matrix(y_test, y_pred_de_re))
# fpr, tpr, thresh = roc_curve(y_test, y_pred_de)
auc_score_de_re = roc_auc_score(y_test, y_pred_de_re)
print("auc_score_de_re:",auc_score_de_re)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_de_re))